In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt

from markovBike.manager.manager import Manager
from markovBike.data_source.source import database_queries, get_stations_data
from markovBike.model.model import calculate_probability_matrix, BSiBSj, BSiBSj2, BSiTBj, TBiTBj

In [ ]:
verbose = True

n_stations = 2_000

n_trips = 1_000_000

In [ ]:
trips_raw = get_stations_data(database_queries(n_trips)['trips'], verbose=True)

probability_matrix = calculate_probability_matrix(trips_raw)

Manager.transition_matrix_heatmaps(probability_matrix)

In [ ]:
trips_raw = get_stations_data(database_queries(1_000_000)['trips'], verbose=True)

In [ ]:
test = calculate_probability_matrix(trips_raw)

In [ ]:
test.shape

In [ ]:
import numpy as np

np.unique(test)